<a href="https://colab.research.google.com/github/OtolaHan/Otola_Han_KAAN/blob/main/weighted2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install pymorphy2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 kB 646.8 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 29.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13706 sha256=25c073feca279193227f9f131cc1c17dc30cf5a10e337442b8d09027712ee7d7
  Stored in directory: /root/.cache/pip/wheels/fc/ab/d4/5da2067ac95b36618c629a5f93f809425700506f72c9732fac
Successfully built docopt


In [3]:
import pandas as pd
import pymorphy2
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split

# Загрузка тонального словаря LinisCrowd 2015
tone_dict = pd.read_csv('/content/words_all_full_rating.csv', sep=';')

# Функция для вычисления признаков на основе тонального словаря
def calculate_tone_features(text):
    tokens = text.split()
    tone_values = [tone_dict[tone_dict['Words'] == token]['average rate'].values[0] for token in tokens if token in tone_dict['Words'].values]
    if len(tone_values) > 0:
        return [
            sum(tone_values) / len(tone_values),  # X1 - среднее значений тональностей слов
            max(tone_values),  # X2 - максимальное значение тональностей слов
            min(tone_values),  # X3 - минимальное значение тональностей слов
            sum(tone_values),  # X4 - суммарное значение тональностей слов
            sum(value > 0 for value in tone_values),  # X5 - количество положительных значений тональностей слов
            sum(value < 0 for value in tone_values)  # X6 - количество отрицательных значений тональностей слов
        ]
    else:
        return [0] * 6

# Инициализация анализатора
morph = pymorphy2.MorphAnalyzer()

# Функция для вычисления морфологических признаков
def calculate_morphological_features(text):
    tokens = text.split()
    pos_counts = {
        'NOUN': 0.3,
        'ADJF': 0.2,
        'ADJS': 0,
        'VERB': 0,
        'INFN': 0,
        'GRND': 0,
        'NUMR': 0,
        'ADVB': 0,
        'NPRO': 0.1,
        'PRED': 0,
        'PREP': 0,
        'CONJ': 0.1,
        'PRCL': 0,
        'INTJ': 0
    }
    total_tokens = 0

    for token in tokens:
        parsed = morph.parse(token)[0]
        pos = parsed.tag.POS
        if pos in pos_counts:
            pos_counts[pos] += 1
        total_tokens += 1

    features = []
    for pos, count in pos_counts.items():
        features.append(count / total_tokens if total_tokens > 0 else 0)

    return features

# Загрузка данных из датасетов negative и positive
negative_data = pd.read_csv('/content/negative.csv')
positive_data = pd.read_csv('/content/positive.csv')

# Объединение данных и добавление меток классов
data = pd.concat([negative_data, positive_data], ignore_index=True)
data['ttype'] = data['ttype'].map({-1: -1, 1: 1})

# Вычисление признаков на основе тонального словаря и морфологических признаков
data['tone_features'] = data['ttext'].apply(calculate_tone_features)
data['morphological_features'] = data['ttext'].apply(calculate_morphological_features)

# Разделение данных на обучающую и тестовую выборки
train_data, test_data = train_test_split(data, test_size=0.025, random_state=42)

# Формирование X_train_Final, y_train_Final, X_test_Final
X_train_Final = pd.concat([pd.DataFrame(train_data['tone_features'].tolist()), pd.DataFrame(train_data['morphological_features'].tolist())], axis=1)
y_train_Final = train_data['ttype']
X_test_Final = pd.concat([pd.DataFrame(test_data['tone_features'].tolist()), pd.DataFrame(test_data['morphological_features'].tolist())], axis=1)

# Сохранение файлов X_train_Final, y_train_Final, X_test_Final
X_train_Final.to_csv('X_train_Final.csv', index=False, header=True)
y_train_Final.to_csv('y_train_Final.csv', index=False, header=True)
X_test_Final.to_csv('X_test_Final.csv', index=False, header=True)

# Обучение модели
model = LogisticRegression()
model.fit(X_train_Final.values, y_train_Final.values)

# Предсказание на тестовых данных
y_pred = model.predict(X_test_Final)

# Загрузить данные из файла X_test_Final.csv
X_test = pd.read_csv('X_test_Final.csv')

# Взять первые 50 строк из X_test для формирования тестового набора X_test_50
X_test_50 = X_test.sample(n=50, random_state=42)

# Сформировать предсказания для X_test_50
y_pred = model.predict(X_test_50.values)

# Сохранить предсказания в файл sentiment.csv
pd.DataFrame(y_pred, columns=['ttype']).to_csv('sentiment.csv', index=False, header=False)

# Оценка качества модели с помощью weighted F1-score
y_test = test_data['ttype'].iloc[:50]
f1 = f1_score(y_test, y_pred, average='weighted')
print(f"Weighted F1-score: {f1:.4f}")

Weighted F1-score: 0.4929


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
